In [179]:
import json
import pickle
import re
import collections
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from random import randint
import sys
import math

In [180]:
# vocabulary?
filenames = [f for f in listdir(ff_path) if isfile(join(ff_path, f))]
candidate_list = []  # use this to test my PLSA model

for file in filenames:
    try:
        with open(ff_path + file,'rb') as f:
            candidate = pickle.load(f)
            
            # too big to process, sampling 1000 followers from each representative nodes
            for c in candidate[:1000]:
                if c not in candidate_list:
                    candidate_list.append(c)
           
    except EOFError:
        print(file)
        pass

KeyboardInterrupt: 

In [ ]:
with open (path + 'unique.pickle','wb+') as f:
    pickle.dump(candidate_list,f)

In [ ]:
with open (path + 'unique.pickle','rb') as f:
    candidate_list = pickle.load(f)

In [ ]:
# open features
with open (path + 'represent_id.pickle','rb') as f:
    feature = pickle.load(f)

In [ ]:
len(candidate_list)

# construct a vector

In [ ]:
def construct_vector(candidate_list,feature):
    V = dict((c,np.zeros(len(feature),)) for c in candidate_list)

    for i in range(len(feature)):  
        with open(ff_path + str(feature[i]) + '.pickle','rb') as f:

                candidate = pickle.load(f)

                for c in candidate:
                    if c in V.keys():
                        #print('true')
                        V[c][i] = 1 
    return V

In [ ]:
Vector = construct_vector(candidate_list,feature)

# test block to see how sparse the matrix is

In [ ]:
def test_sparsity(V,feature_num):
    for v in list(V.keys()):
        total_feature = np.sum(V[v])
        if total_feature >= feature_num:
            print('user_id:', v,'\t\ttotal feature:', total_feature)
    return None

In [ ]:
test_sparsity(Vector,40)

# estimate prior
<img src= 'prior.png'>

i feel like this prior wont affect ranking so can be omit, doesnt change with Ux

In [ ]:
with open(path + 'prob_following_rep.pickle','rb') as f:
    prob = pickle.load(f)

In [ ]:
# prob[1]

# Build Classifier
<img src=untitled.png>

In [181]:
def rank_candidate(Vector, feature):
    candidate_score = {}

    for v in list(V.keys()):  # for each user
        
        score = 0
        for i in range(len(feature)): # for each feature            

            if V[v][i] == 1:  # if follow
                score += math.log2(prob[0][i]) # ln(p(follow)) 

            elif V[v][i] == 0:
                score += math.log2(prob[1][i]) # ln(p(unfollow))

        candidate_score[v] = score
    
    candidate_score_ranked = sorted(candidate_score.items(),key=lambda x:x[1],reverse=True)
    
    return candidate_score_ranked

In [182]:
c_rank = rank_candidate(V, feature)
c_rank[:10]

[(781149191808712704, -1758.2236764613422),
 (791068282736484352, -1762.922426437889),
 (785599640146108417, -1842.8592383852822),
 (779659530242142208, -1844.5119806159519),
 (787488684912156672, -1867.8320771088486),
 (754013832108994560, -1871.9273272390972),
 (780206098431213568, -1872.5019793483407),
 (550557628, -1876.569656086811),
 (788347210698256384, -1882.9656627997808),
 (790322349098749952, -1890.5031798600978)]

In [183]:
n_f = []
for c in c_rank:
    n_f.append(np.sum(V[c[0]]))
n_f[:10]

[46.0, 49.0, 35.0, 34.0, 32.0, 30.0, 31.0, 31.0, 34.0, 31.0]

In [184]:
with open (path + 'ranked_candidate_id.pickle', 'wb+') as f:
    pickle.dump(c_rank,f)

# questions: 

1. for initial list:
I applied very stringent approach (only look at user profile who provides 'chicago' as their location). Reasonable? Extra learning required?

2. feature selection:
How many representative nodes do we need?

3. feature ranking:
My approach reasonable? Any improvement?

4. rate limit problem
when use the clf on test data, need to retrieve test nodes's friends_list. thus only can test a small portion once a time.
can't be real time very slow

5. is this approach generic enough? 
chicago is famous. What if it's non-famous? what if it's smaller or bigger area?

6. How to utilize the query user by keyword?

7. How to test the performance? 
We hae ground truth (a small portion of user will list their profile location <10%)

# Things to do:
a. SVD rank feature 

b. haven\'t add prior to naive bayes

c. think!

I have different goal than Amin's approach.
--------------------------------------------------------------------------------------------------------
representatives:
(1) Amin's approach is very time consuming, might hit a lot of rate limit. imagine loop over 150,000+ people, query their followers to get the "follower number"
(2) filter the top 200 frequent "TF" potential candidate by their location? might sacrifice some representatives that doesn't explicitly list their location? Still consuming, hit rate limit 

Difference of Amin's goal and my goal
he wants to find interest, i want to find location. I have more information than he has. How can I utilize these information.